# Yapay Zeka ile Görüntü Sınıflandırma Projesi
Bu notebook, 10 farklı hayvan sınıfını sınıflandırmak için bir CNN modeli oluşturur ve çeşitli veri manipülasyon teknikleriyle dayanıklılığı test eder.

## Gerekli Kütüphanelerin İmport Edilmesi

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1. Veri Setinin Yüklenmesi ve Hazırlanması

In [ ]:
# Resimlerin bulunduğu ana dizini belirlemek
image_dir = "C:\\Users\\omers\\Desktop\\dataset\\JPEGImages"

# 10 sınıfın adlarını girmek
classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant panda', 'polar bear']

# Resimleri ve etiketlerini yüklemek
def load_images(image_dir, classes, image_size=(128, 128)):
    images = []
    labels = []
    for label, class_name in enumerate(classes):
        class_dir = os.path.join(image_dir, class_name)
        for image_name in os.listdir(class_dir)[:650]:
            image_path = os.path.join(class_dir, image_name)
            image = Image.open(image_path)
            image = image.resize(image_size)
            image = np.array(image) / 255.0
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

# Veriyi yüklemek
X, y = load_images(image_dir, classes)

# Eğitimi ve test setlerini ayırmak
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")

## 2. Veri Artırma (Augmentation)

In [ ]:
# Eğitim verisi için veri artırma (augmentation) kullanmak
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

## 3. CNN Modelinin Tasarlanması ve Eğitilmesi

In [ ]:
# Modeli oluşturmak
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(classes), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

## 4. Model Performansını Test Etme

In [ ]:
# Modeli test et
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test seti doğruluğu: {test_acc * 100:.2f}%")

## 5. Manipüle Edilmiş Veri ile Test

In [ ]:
# Manipüle edilmiş test seti ile modeli test et
def get_manipulated_images(image):
    return cv2.convertScaleAbs(image, alpha=1.2, beta=50)

X_test_manipulated = np.array([get_manipulated_images(img) for img in X_test])
test_loss, test_acc = model.evaluate(X_test_manipulated, y_test)
print(f"Manipüle edilmiş test seti doğruluğu: {test_acc * 100:.2f}%")